In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('atlantis_citizens_final')

In [3]:
# 1) Data Pre Processing
# a) Data Imputation
# checking null  values
df.isnull().sum()

Citizen_ID             0
Diet_Type              0
District_Name          0
Occupation             0
Wealth_Index        1055
House_Size_sq_ft    1197
Life_Expectancy      614
Vehicle_Owned          0
Work_District          0
Bio_Hash               0
dtype: int64

In [4]:
# diffrentiating between numerical and categorical columns
num_cols = [
    "Wealth_Index",
    "House_Size_sq_ft",
    "Life_Expectancy"
]
cat_cols = [
    "Diet_Type",
    "District_Name",
    "Vehicle_Owned",
    "Work_District"
]

In [5]:
from sklearn.impute import SimpleImputer
# replacing null values with mean
num_imputer = SimpleImputer(strategy="mean")
df[num_cols] = num_imputer.fit_transform(df[num_cols])
# replacing null values with most frequent 
cat_imputer = SimpleImputer(strategy="most_frequent")
df[cat_cols] = cat_imputer.fit_transform(df[cat_cols])

In [6]:
# b) Categorical Feature Enconding
# splitting features and targe
drop_cols = ["Citizen_ID", "Bio_Hash"] # since citizen_id and bio hash can not be used for ml model
x = df.drop(columns=drop_cols + ["Occupation"])
y = df["Occupation"]
# encoding target
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
le = LabelEncoder()
y = le.fit_transform(y)
# one hot encoding 
from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", "passthrough", num_cols)
    ]
)

In [8]:
# 2) Building ML Model(Random Forest)
# Building and Training Random Forest Model
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=12,
    min_samples_leaf=4,
    min_samples_split=10,
    max_features="sqrt",
    random_state=42,
    n_jobs=1,
    class_weight="balanced"
)

In [9]:
# Pipeline 
from sklearn.pipeline import Pipeline
pipeline = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("model", rf)
    ]
)
pipeline.fit(x, y)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocess', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('cat', ...), ('num', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers cont

In [10]:
# start of hyperparameter tuning
# making grid
param_distributions = {
    "model__n_estimators": [200, 300, 400],
    "model__max_depth": [8, 10, 12, 14],
    "model__min_samples_leaf": [2, 4, 6],
    "model__max_features": ["sqrt", 0.5, 0.7]
}


In [11]:
# finding best paramters for the model
from sklearn.model_selection import RandomizedSearchCV
search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_distributions,
    n_iter=12,                
    scoring="f1_macro",
    cv=3,
    n_jobs=1,                
    random_state=42,
    verbose=3
)
search.fit(x, y)
best_model = search.best_estimator_

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV 1/3] END model__max_depth=12, model__max_features=0.7, model__min_samples_leaf=4, model__n_estimators=400;, score=0.608 total time=   7.9s
[CV 2/3] END model__max_depth=12, model__max_features=0.7, model__min_samples_leaf=4, model__n_estimators=400;, score=0.607 total time=   7.9s
[CV 3/3] END model__max_depth=12, model__max_features=0.7, model__min_samples_leaf=4, model__n_estimators=400;, score=0.610 total time=   7.9s
[CV 1/3] END model__max_depth=8, model__max_features=0.5, model__min_samples_leaf=2, model__n_estimators=300;, score=0.604 total time=   3.4s
[CV 2/3] END model__max_depth=8, model__max_features=0.5, model__min_samples_leaf=2, model__n_estimators=300;, score=0.609 total time=   3.4s
[CV 3/3] END model__max_depth=8, model__max_features=0.5, model__min_samples_leaf=2, model__n_estimators=300;, score=0.602 total time=   3.4s
[CV 1/3] END model__max_depth=8, model__max_features=sqrt, model__min_samples_leaf=4

In [12]:
# showing best parameters
print(search.best_params_)

{'model__n_estimators': 200, 'model__min_samples_leaf': 2, 'model__max_features': 0.7, 'model__max_depth': 10}


In [13]:
# Loading test data
test_df = pd.read_csv("test_atlantis_hidden.")
citizen_ids = test_df["Citizen_ID"]
test_df

,Citizen_ID,Diet_Type,District_Name,Wealth_Index,House_Size_sq_ft,Life_Expectancy,Vehicle_Owned,Work_District,Bio_Hash
0,CIT_15383,Seaweed,Mariana Plaza,2851,453,53,Sea Scooter,Coral Slums,0392e7e1
1,CIT_14830,Seaweed,Mariana Plaza,5176,815,58,No Vehicle,Deep Trench,da47b527
2,CIT_17388,Exotic Imports,Deep Trench,3772,390,29,Fin Bicycle,The Golden Reef,970f8714
3,CIT_17438,Seaweed,Deep Trench,1288,131,44,No Vehicle,Deep Trench,7ca70526
4,CIT_16735,Seafood,Deep Trench,1736,182,43,Sea Scooter,The Golden Reef,c85a5d8b
...,...,...,...,...,...,...,...,...,...
3933,CIT_15659,Exotic Imports,Mariana Plaza,3537,572,50,Fin Bicycle,Mariana Plaza,8b662009
3934,CIT_16061,Seafood,Coral Slums,1769,116,41,Sea Scooter,Deep Trench,7d7f979a
3935,CIT_17913,Seaweed,The Golden Reef,7972,1689,72,No Vehicle,Coral Slums,f182c545
3936,CIT_17666,Exotic Imports,Coral Slums,1904,138,34,Sea Scooter,Coral Slums,9ab7b011


In [14]:
# splitting features and target
drop_cols = ["Citizen_ID", "Bio_Hash"] # since citizen_id, bio hash and diet type can not be used for ml model
x_test = test_df.drop(columns=drop_cols)

In [15]:
# Make predictions
y_pred = predictions = best_model.predict(x_test)
y_pred = le.inverse_transform(y_pred)

In [17]:
# making submission file
submission = pd.DataFrame({
    "Citizen_ID": citizen_ids,
    "Occupation": y_pred
})
# mapping it according to convention
occupation_mapping = {
    'Fisher': 2,
    'Miner': 3, 
    'Scribe': 4, 
    'Warrior': 0, 
    'Merchant':1 } 
submission['Occupation'] = submission['Occupation'].map(occupation_mapping)
submission.to_csv("sample52.csv", index=False)